# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:03:56,134 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedCbo2002Ocupacao','compliance_cbo2002ocupacao')

2024-09-25 19:04:00,931 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedCbo2002Ocupacao)
2024-09-25 19:04:49,601 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_cbo2002ocupacao
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:04:49,614 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:04:49,615 - INFO - 
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_cbo2002ocupacao

2024-09-25 19:04:49,701 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedCbo2002Ocupacao'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:04:49,717 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:04:58,181 - INFO - Dados persistidos com sucesso
2024-09-25 19:04:58,182 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedCbo2002Ocupacao
2024-09-25 19:04:58,182 - INFO - Aguarde... Realizando optimize
2024-09-25 19:05:01,108 - INFO - Optimize executado com sucesso.
2024-09-25 19:05:01,109 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:05:28,651 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:05:28,777 - INFO - Sessão Spark finalizada.
